In [1]:
import json
import pandas as pd
import requests
import datetime
import time

In [2]:
start_date = datetime.date(2021, 8, 1)
end_date = datetime.date(2022, 1, 31)
topic = 'Microsoft'
ticker = 'MSFT'

In [3]:
def GetNewsArticlesDataFrame(begin, end, topic):
    try:
        base_url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'

        query = {'api-key':'9AWe7aBHew0w38buCynHFQr9spBC95ho', 
                'begin_date': begin,
                'end_date': end,
                'fq' : 'news_desk:("Business")'
                'q' : topic
                }
        response = requests.get(base_url, params=query)
        
        if (response.status_code == 200):
            df = pd.DataFrame(response.json()['response']['docs'])
            #df = df.drop(columns=['snippet', 'lead_paragraph', 'multimedia', 'document_type', 'news_desk', 'uri', '_id'])
            df['Current Date'] = begin
            df.set_index('Current Date', inplace = True)
            return df
        else:
            print("error...", response.status_code)
        return pd.DataFrame()
    except requests.exceptions.HTTPError as error:
        print(error)

In [4]:
# delta time
delta = datetime.timedelta(days=1)

begin_date = start_date
next_date = begin_date + delta
begin_date += delta

# Initial Dataframe
news_df = GetNewsArticlesDataFrame(begin_date, next_date, topic)
  
# iterate over range of dates
while (begin_date < end_date):
    next_date = begin_date + delta

    temp_df = GetNewsArticlesDataFrame(begin_date, next_date, topic)

    if (not temp_df.empty):
        news_df = news_df.append(temp_df, ignore_index=False)

    begin_date += delta

    # Suggested by NyTimes API to sleep for 6 seconds before next call
    time.sleep(6)

In [5]:
news_df.shape

(1709, 20)

In [6]:
news_df.dtypes

abstract            object
web_url             object
snippet             object
lead_paragraph      object
print_section       object
print_page          object
source              object
multimedia          object
headline            object
keywords            object
pub_date            object
document_type       object
news_desk           object
section_name        object
byline              object
type_of_material    object
_id                 object
word_count           int64
uri                 object
subsection_name     object
dtype: object

In [7]:
news_df.head()

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,headline,keywords,pub_date,document_type,news_desk,section_name,byline,type_of_material,_id,word_count,uri,subsection_name
Current Date,,,,,,,,,,,,,,,,,,,,
2021-08-02,A hearing officer for the National Labor Relat...,https://www.nytimes.com/2021/08/02/business/am...,A hearing officer for the National Labor Relat...,A hearing officer of the National Labor Relati...,B,1,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Labor Board Official Backs Union Cha...,"[{'name': 'subject', 'value': 'Organized Labor...",2021-08-02T23:25:23+0000,article,Business,Business Day,"{'original': 'By Noam Scheiber', 'person': [{'...",News,nyt://article/0678007c-623f-5a87-b163-8e9b7d18...,665,nyt://article/0678007c-623f-5a87-b163-8e9b7d18...,NaN
2021-08-02,Equinox and Walmart are requiring some employe...,https://www.nytimes.com/2021/08/02/business/va...,Equinox and Walmart are requiring some employe...,With the coronavirus spreading across the coun...,NaN,NaN,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","{'main': 'Home Depot, SoulCycle and other busi...","[{'name': 'subject', 'value': 'Coronavirus Reo...",2021-08-02T12:37:16+0000,article,Business,Business Day,"{'original': 'By Lauren Hirsch', 'person': [{'...",News,nyt://article/0be01f56-c16f-5ba1-80a7-5ffc1e57...,676,nyt://article/0be01f56-c16f-5ba1-80a7-5ffc1e57...,NaN
2021-08-02,"As workers return to the financial district, l...",https://www.nytimes.com/2021/08/02/business/wa...,"As workers return to the financial district, l...",The suits are returning to the office. In chin...,B,1,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'A Wall Street Dressing Down: Always....,"[{'name': 'subject', 'value': 'Banking and Fin...",2021-08-02T07:00:34+0000,article,Business,Business Day,"{'original': 'By Lananh Nguyen, Melodie Jeng a...",News,nyt://article/11b69130-0dc1-5afa-b409-8432369d...,1164,nyt://article/11b69130-0dc1-5afa-b409-8432369d...,NaN
2021-08-02,Financing and security issues are clouding new...,https://www.nytimes.com/2021/08/02/business/gr...,Financing and security issues are clouding new...,"LONDON — A few years ago, Britain agreed to le...",B,3,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Britain Rethinks Letting China Enter...,"[{'name': 'subject', 'value': 'Nuclear Energy'...",2021-08-02T18:14:31+0000,article,Business,Business Day,"{'original': 'By Stanley Reed', 'person': [{'f...",News,nyt://article/22f30812-fe29-55c2-aa1b-70fd2eb9...,1056,nyt://article/22f30812-fe29-55c2-aa1b-70fd2eb9...,NaN
2021-08-02,Corporate requirements for vaccinations have s...,https://www.nytimes.com/2021/08/02/business/wo...,Corporate requirements for vaccinations have s...,The tide has begun to turn on corporate vaccin...,B,5,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",{'main': 'Workplace vaccine mandates reveal a ...,"[{'name': 'subject', 'value': 'Vaccination and...",2021-08-02T12:37:17+0000,article,Business,Business Day,"{'original': 'By Lauren Hirsch', 'person': [{'...",News,nyt://article/334c4dd4-dc62-5710-bdf7-b2752229...,435,nyt://article/334c4dd4-dc62-5710-bdf7-b2752229...,NaN


In [8]:
news_df = news_df.drop(columns=['snippet', 
                                'lead_paragraph', 
                                'multimedia', 
                                'document_type', 
                                'news_desk', 
                                'uri', 
                                '_id', 
                                'uri', 
                                'print_section',
                                'print_page',
                                'subsection_name'
                                ])

In [9]:
news_df.head()

,abstract,web_url,source,headline,keywords,pub_date,section_name,byline,type_of_material,word_count
Current Date,,,,,,,,,,
2021-08-02,A hearing officer for the National Labor Relat...,https://www.nytimes.com/2021/08/02/business/am...,The New York Times,{'main': 'Labor Board Official Backs Union Cha...,"[{'name': 'subject', 'value': 'Organized Labor...",2021-08-02T23:25:23+0000,Business Day,"{'original': 'By Noam Scheiber', 'person': [{'...",News,665
2021-08-02,Equinox and Walmart are requiring some employe...,https://www.nytimes.com/2021/08/02/business/va...,The New York Times,"{'main': 'Home Depot, SoulCycle and other busi...","[{'name': 'subject', 'value': 'Coronavirus Reo...",2021-08-02T12:37:16+0000,Business Day,"{'original': 'By Lauren Hirsch', 'person': [{'...",News,676
2021-08-02,"As workers return to the financial district, l...",https://www.nytimes.com/2021/08/02/business/wa...,The New York Times,{'main': 'A Wall Street Dressing Down: Always....,"[{'name': 'subject', 'value': 'Banking and Fin...",2021-08-02T07:00:34+0000,Business Day,"{'original': 'By Lananh Nguyen, Melodie Jeng a...",News,1164
2021-08-02,Financing and security issues are clouding new...,https://www.nytimes.com/2021/08/02/business/gr...,The New York Times,{'main': 'Britain Rethinks Letting China Enter...,"[{'name': 'subject', 'value': 'Nuclear Energy'...",2021-08-02T18:14:31+0000,Business Day,"{'original': 'By Stanley Reed', 'person': [{'f...",News,1056
2021-08-02,Corporate requirements for vaccinations have s...,https://www.nytimes.com/2021/08/02/business/wo...,The New York Times,{'main': 'Workplace vaccine mandates reveal a ...,"[{'name': 'subject', 'value': 'Vaccination and...",2021-08-02T12:37:17+0000,Business Day,"{'original': 'By Lauren Hirsch', 'person': [{'...",News,435


In [10]:
!pip install yfinance

In [11]:
## Grab Stock price

import yfinance as yf

tickers = [ticker]
stock_df = yf.download(tickers, start=start_date,end = end_date, interval='1d')
stock_df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-08-02,286.359985,286.769989,283.739990,284.820007,283.171143,16267400
2021-08-03,285.420013,287.230011,284.000000,287.119995,285.457794,17879000
2021-08-04,286.220001,287.589996,284.649994,286.510010,284.851349,16191300
2021-08-05,286.880005,289.630005,286.100006,289.519989,287.843903,13900200
2021-08-06,288.510010,289.500000,287.619995,289.459991,287.784271,16589300
...,...,...,...,...,...,...
2022-01-24,292.200012,297.109985,276.049988,296.369995,295.758453,85731500
2022-01-25,291.519989,294.989990,285.170013,288.489990,287.894714,72848600
2022-01-26,307.989990,308.500000,293.029999,296.709991,296.097748,90428900


In [12]:
data_df = news_df.join(stock_df[['Close']])
data_df = data_df.dropna()

In [13]:
data_df.shape

(1267, 11)

In [14]:
data_df.to_csv("./data/{0}.csv".format(topic))